In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
stroke_data = pd.read_csv("healthcare-dataset-stroke-data.csv")

In [3]:
stroke_data.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
stroke_data.shape

(5110, 12)

In [5]:
stroke_data.isna().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

# K-Means Clustering for Imputation

In [6]:
training_data = stroke_data.drop(["bmi", "stroke", "id"], axis=1)

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
training_data[["gender"]] = training_data[["gender"]].apply(le.fit_transform)
training_data[["ever_married"]] = training_data[["ever_married"]].apply(le.fit_transform)
training_data[["work_type"]] = training_data[["work_type"]].apply(le.fit_transform)
training_data[["Residence_type"]] = training_data[["Residence_type"]].apply(le.fit_transform)
training_data[["smoking_status"]] = training_data[["smoking_status"]].apply(le.fit_transform)

In [8]:
from sklearn.cluster import KMeans

km = KMeans(n_clusters=5)

In [9]:
km.fit(training_data)

KMeans(n_clusters=5)

In [10]:
km.labels_

array([1, 1, 4, ..., 3, 1, 0], dtype=int32)

In [11]:
km.cluster_centers_

array([[3.95596052e-01, 5.91860289e+01, 1.26803341e-01, 6.90964313e-02,
        9.09643128e-01, 1.89597570e+00, 5.14047077e-01, 7.51724829e+01,
        1.52619590e+00],
       [4.75000000e-01, 6.04875000e+01, 2.57812500e-01, 1.60937500e-01,
        8.85937500e-01, 1.97031250e+00, 5.14062500e-01, 2.10341359e+02,
        1.57968750e+00],
       [4.40322581e-01, 2.24016774e+01, 2.25806452e-02, 3.22580645e-03,
        3.33870968e-01, 2.54516129e+00, 4.79032258e-01, 1.25381790e+02,
        1.14516129e+00],
       [4.02230971e-01, 2.09310761e+01, 1.57480315e-02, 6.56167979e-04,
        3.01181102e-01, 2.52099738e+00, 5.19685039e-01, 8.04562730e+01,
        1.12729659e+00],
       [4.02378593e-01, 5.79187314e+01, 1.26858276e-01, 7.82953419e-02,
        9.13776016e-01, 1.88206145e+00, 4.96531219e-01, 1.07474658e+02,
        1.57284440e+00]])

In [12]:
stroke_data["target"] = km.predict(training_data)

In [13]:
X = stroke_data.dropna()[["target"]]
y = stroke_data.dropna()[["bmi"]]

In [14]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

lr.fit(X,y)

LinearRegression()

In [15]:
test_data = stroke_data[stroke_data.isnull().any(axis=1)][["target"]]

In [16]:
pred = lr.predict(test_data)

In [17]:
stroke_data.loc[stroke_data.isnull().any(axis=1),"bmi"] = pred

In [18]:
stroke_data_cat = stroke_data.select_dtypes(object)
stroke_data_num = stroke_data.select_dtypes(np.number)

In [19]:
stroke_data_cat.head()

,gender,ever_married,work_type,Residence_type,smoking_status
0,Male,Yes,Private,Urban,formerly smoked
1,Female,Yes,Self-employed,Rural,never smoked
2,Male,Yes,Private,Rural,never smoked
3,Female,Yes,Private,Urban,smokes
4,Female,Yes,Self-employed,Rural,never smoked


In [20]:
stroke_data_num.head()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,target
0,9046,67.0,0,1,228.69,36.60000,1,1
1,51676,61.0,0,0,202.21,29.58672,1,1
2,31112,80.0,0,1,105.92,32.50000,1,4
3,60182,49.0,0,0,171.23,34.40000,1,1
4,1665,79.0,1,0,174.12,24.00000,1,1


In [21]:
stroke_data_num[['age']] = pd.cut(stroke_data_num['age'], bins=[0,35,60,100], labels=['Youth', 'Mid_Age', "Old_Age"])

In [22]:
stroke_data_num[['avg_glucose_level']] = pd.cut(stroke_data_num['avg_glucose_level'], bins=[0, 100, 160, 220, 300], labels=['Low_Glucose', 'Non_Diabetic', "High_Glucose", "Diabetic"])

In [23]:
stroke_data_num[['bmi']] = pd.cut(stroke_data_num['bmi'], bins=[0, 18.5, 25, 30, 100], labels=['Underweight', 'Healthy', "Overweight", "Obese"])

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [25]:
stroke_data_num[['bmi']] = stroke_data_num[['bmi']].apply(le.fit_transform)
stroke_data_num[['avg_glucose_level']] = stroke_data_num[['avg_glucose_level']].apply(le.fit_transform)
stroke_data_num[['age']] = stroke_data_num[['age']].apply(le.fit_transform)

In [26]:
stroke_data_cat = stroke_data_cat.apply(le.fit_transform)

In [27]:
stroke_data_final = pd.concat([stroke_data_num, stroke_data_cat], axis=1)
stroke_data_final.head()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,target,gender,ever_married,work_type,Residence_type,smoking_status
0,9046,1,0,1,0,1,1,1,1,1,2,1,1
1,51676,1,0,0,1,2,1,1,0,1,3,0,2
2,31112,1,0,1,3,1,1,4,1,1,2,0,2
3,60182,0,0,0,1,1,1,1,0,1,2,1,3
4,1665,1,1,0,1,0,1,1,0,1,3,0,2


In [28]:
X = stroke_data_final.drop(['id', 'stroke'], axis=1)
y = stroke_data_final[['stroke']]

In [34]:
X = X.to_numpy()
y = y.to_numpy()

# XGBoost

In [82]:
prm_xgb = {
    'booster': 'gbtree',
    'objective': 'binary:logistic',
    'max_depth': 5,
    'learning_rate': 0.1,
    'colsample_bytree': 0.9,
    'subsample': 0.9,
    'eval_metric': 'rmse',
}

In [57]:
# np.random.seed(20161218)
# flg_train = np.random.choice([False, True], len(y), p=[0.3, 0.7])
# flg_valid = np.logical_not(flg_train)

In [58]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [59]:
import xgboost as xgb
dt_xgb   = xgb.DMatrix(x_train, y_train)
dv_xgb   = xgb.DMatrix(x_test, y_test)

In [60]:
import time
time_s = time.time()
obj_xgb = xgb.train(
    prm_xgb, dt_xgb, 10000,
    [(dt_xgb, 'train'), (dv_xgb, 'valid')])
time_t = time.time()
print(time_t - time_s)

[0]	train-rmse:0.45922	valid-rmse:0.45947
[1]	train-rmse:0.42350	valid-rmse:0.42422
[2]	train-rmse:0.39219	valid-rmse:0.39352
[3]	train-rmse:0.36480	valid-rmse:0.36664
[4]	train-rmse:0.34094	valid-rmse:0.34298
[5]	train-rmse:0.31999	valid-rmse:0.32282
[6]	train-rmse:0.30194	valid-rmse:0.30549
[7]	train-rmse:0.28630	valid-rmse:0.29061
[8]	train-rmse:0.27282	valid-rmse:0.27743
[9]	train-rmse:0.26133	valid-rmse:0.26664
[10]	train-rmse:0.25148	valid-rmse:0.25742
[11]	train-rmse:0.24323	valid-rmse:0.24962
[12]	train-rmse:0.23616	valid-rmse:0.24333
[13]	train-rmse:0.23028	valid-rmse:0.23785
[14]	train-rmse:0.22518	valid-rmse:0.23312
[15]	train-rmse:0.22102	valid-rmse:0.22935
[16]	train-rmse:0.21735	valid-rmse:0.22612
[17]	train-rmse:0.21443	valid-rmse:0.22361
[18]	train-rmse:0.21195	valid-rmse:0.22175
[19]	train-rmse:0.20993	valid-rmse:0.21999
[20]	train-rmse:0.20824	valid-rmse:0.21846
[21]	train-rmse:0.20680	valid-rmse:0.21701
[22]	train-rmse:0.20548	valid-rmse:0.21599
[23]	train-rmse:0.204

[189]	train-rmse:0.17528	valid-rmse:0.22384
[190]	train-rmse:0.17515	valid-rmse:0.22391
[191]	train-rmse:0.17503	valid-rmse:0.22397
[192]	train-rmse:0.17493	valid-rmse:0.22405
[193]	train-rmse:0.17488	valid-rmse:0.22410
[194]	train-rmse:0.17475	valid-rmse:0.22427
[195]	train-rmse:0.17452	valid-rmse:0.22442
[196]	train-rmse:0.17443	valid-rmse:0.22430
[197]	train-rmse:0.17434	valid-rmse:0.22432
[198]	train-rmse:0.17432	valid-rmse:0.22428
[199]	train-rmse:0.17418	valid-rmse:0.22445
[200]	train-rmse:0.17408	valid-rmse:0.22452
[201]	train-rmse:0.17389	valid-rmse:0.22465
[202]	train-rmse:0.17377	valid-rmse:0.22473
[203]	train-rmse:0.17359	valid-rmse:0.22483
[204]	train-rmse:0.17353	valid-rmse:0.22486
[205]	train-rmse:0.17346	valid-rmse:0.22481
[206]	train-rmse:0.17333	valid-rmse:0.22498
[207]	train-rmse:0.17321	valid-rmse:0.22506
[208]	train-rmse:0.17313	valid-rmse:0.22513
[209]	train-rmse:0.17294	valid-rmse:0.22535
[210]	train-rmse:0.17288	valid-rmse:0.22539
[211]	train-rmse:0.17282	valid-r

[376]	train-rmse:0.16025	valid-rmse:0.23608
[377]	train-rmse:0.16023	valid-rmse:0.23608
[378]	train-rmse:0.16021	valid-rmse:0.23614
[379]	train-rmse:0.16019	valid-rmse:0.23610
[380]	train-rmse:0.16009	valid-rmse:0.23612
[381]	train-rmse:0.16008	valid-rmse:0.23621
[382]	train-rmse:0.16007	valid-rmse:0.23607
[383]	train-rmse:0.16003	valid-rmse:0.23619
[384]	train-rmse:0.15993	valid-rmse:0.23630
[385]	train-rmse:0.15984	valid-rmse:0.23651
[386]	train-rmse:0.15972	valid-rmse:0.23653
[387]	train-rmse:0.15969	valid-rmse:0.23649
[388]	train-rmse:0.15959	valid-rmse:0.23655
[389]	train-rmse:0.15961	valid-rmse:0.23665
[390]	train-rmse:0.15958	valid-rmse:0.23664
[391]	train-rmse:0.15955	valid-rmse:0.23666
[392]	train-rmse:0.15952	valid-rmse:0.23662
[393]	train-rmse:0.15950	valid-rmse:0.23664
[394]	train-rmse:0.15945	valid-rmse:0.23659
[395]	train-rmse:0.15941	valid-rmse:0.23653
[396]	train-rmse:0.15926	valid-rmse:0.23675
[397]	train-rmse:0.15922	valid-rmse:0.23683
[398]	train-rmse:0.15908	valid-r

[563]	train-rmse:0.15185	valid-rmse:0.24129
[564]	train-rmse:0.15180	valid-rmse:0.24138
[565]	train-rmse:0.15177	valid-rmse:0.24142
[566]	train-rmse:0.15168	valid-rmse:0.24154
[567]	train-rmse:0.15166	valid-rmse:0.24150
[568]	train-rmse:0.15165	valid-rmse:0.24159
[569]	train-rmse:0.15163	valid-rmse:0.24167
[570]	train-rmse:0.15161	valid-rmse:0.24166
[571]	train-rmse:0.15158	valid-rmse:0.24169
[572]	train-rmse:0.15153	valid-rmse:0.24166
[573]	train-rmse:0.15155	valid-rmse:0.24167
[574]	train-rmse:0.15151	valid-rmse:0.24181
[575]	train-rmse:0.15141	valid-rmse:0.24189
[576]	train-rmse:0.15138	valid-rmse:0.24196
[577]	train-rmse:0.15136	valid-rmse:0.24196
[578]	train-rmse:0.15134	valid-rmse:0.24204
[579]	train-rmse:0.15127	valid-rmse:0.24206
[580]	train-rmse:0.15128	valid-rmse:0.24215
[581]	train-rmse:0.15123	valid-rmse:0.24216
[582]	train-rmse:0.15123	valid-rmse:0.24206
[583]	train-rmse:0.15121	valid-rmse:0.24213
[584]	train-rmse:0.15119	valid-rmse:0.24217
[585]	train-rmse:0.15116	valid-r

[750]	train-rmse:0.14641	valid-rmse:0.24571
[751]	train-rmse:0.14640	valid-rmse:0.24576
[752]	train-rmse:0.14644	valid-rmse:0.24559
[753]	train-rmse:0.14643	valid-rmse:0.24573
[754]	train-rmse:0.14642	valid-rmse:0.24567
[755]	train-rmse:0.14636	valid-rmse:0.24567
[756]	train-rmse:0.14637	valid-rmse:0.24560
[757]	train-rmse:0.14637	valid-rmse:0.24561
[758]	train-rmse:0.14633	valid-rmse:0.24577
[759]	train-rmse:0.14631	valid-rmse:0.24585
[760]	train-rmse:0.14626	valid-rmse:0.24595
[761]	train-rmse:0.14622	valid-rmse:0.24608
[762]	train-rmse:0.14617	valid-rmse:0.24609
[763]	train-rmse:0.14612	valid-rmse:0.24622
[764]	train-rmse:0.14612	valid-rmse:0.24619
[765]	train-rmse:0.14607	valid-rmse:0.24619
[766]	train-rmse:0.14605	valid-rmse:0.24624
[767]	train-rmse:0.14598	valid-rmse:0.24645
[768]	train-rmse:0.14594	valid-rmse:0.24636
[769]	train-rmse:0.14589	valid-rmse:0.24638
[770]	train-rmse:0.14588	valid-rmse:0.24645
[771]	train-rmse:0.14584	valid-rmse:0.24638
[772]	train-rmse:0.14582	valid-r

...

[9173]	train-rmse:0.12361	valid-rmse:0.27268
[9174]	train-rmse:0.12361	valid-rmse:0.27271
[9175]	train-rmse:0.12362	valid-rmse:0.27265
[9176]	train-rmse:0.12362	valid-rmse:0.27260
[9177]	train-rmse:0.12361	valid-rmse:0.27258
[9178]	train-rmse:0.12360	valid-rmse:0.27255
[9179]	train-rmse:0.12361	valid-rmse:0.27240
[9180]	train-rmse:0.12360	valid-rmse:0.27246
[9181]	train-rmse:0.12362	valid-rmse:0.27239
[9182]	train-rmse:0.12363	valid-rmse:0.27213
[9183]	train-rmse:0.12363	valid-rmse:0.27218
[9184]	train-rmse:0.12362	valid-rmse:0.27216
[9185]	train-rmse:0.12363	valid-rmse:0.27204
[9186]	train-rmse:0.12362	valid-rmse:0.27209
[9187]	train-rmse:0.12363	valid-rmse:0.27198
[9188]	train-rmse:0.12363	valid-rmse:0.27199
[9189]	train-rmse:0.12363	valid-rmse:0.27203
[9190]	train-rmse:0.12363	valid-rmse:0.27202
[9191]	train-rmse:0.12363	valid-rmse:0.27206
[9192]	train-rmse:0.12362	valid-rmse:0.27203
[9193]	train-rmse:0.12362	valid-rmse:0.27206
[9194]	train-rmse:0.12362	valid-rmse:0.27212
[9195]	tra

[9356]	train-rmse:0.12358	valid-rmse:0.27291
[9357]	train-rmse:0.12358	valid-rmse:0.27285
[9358]	train-rmse:0.12361	valid-rmse:0.27268
[9359]	train-rmse:0.12361	valid-rmse:0.27264
[9360]	train-rmse:0.12360	valid-rmse:0.27273
[9361]	train-rmse:0.12361	valid-rmse:0.27276
[9362]	train-rmse:0.12362	valid-rmse:0.27263
[9363]	train-rmse:0.12362	valid-rmse:0.27266
[9364]	train-rmse:0.12362	valid-rmse:0.27269
[9365]	train-rmse:0.12361	valid-rmse:0.27266
[9366]	train-rmse:0.12361	valid-rmse:0.27273
[9367]	train-rmse:0.12361	valid-rmse:0.27271
[9368]	train-rmse:0.12361	valid-rmse:0.27267
[9369]	train-rmse:0.12362	valid-rmse:0.27253
[9370]	train-rmse:0.12363	valid-rmse:0.27251
[9371]	train-rmse:0.12362	valid-rmse:0.27262
[9372]	train-rmse:0.12363	valid-rmse:0.27264
[9373]	train-rmse:0.12363	valid-rmse:0.27266
[9374]	train-rmse:0.12362	valid-rmse:0.27267
[9375]	train-rmse:0.12363	valid-rmse:0.27270
[9376]	train-rmse:0.12363	valid-rmse:0.27273
[9377]	train-rmse:0.12363	valid-rmse:0.27268
[9378]	tra

[9539]	train-rmse:0.12355	valid-rmse:0.27218
[9540]	train-rmse:0.12355	valid-rmse:0.27233
[9541]	train-rmse:0.12355	valid-rmse:0.27231
[9542]	train-rmse:0.12354	valid-rmse:0.27243
[9543]	train-rmse:0.12355	valid-rmse:0.27255
[9544]	train-rmse:0.12354	valid-rmse:0.27255
[9545]	train-rmse:0.12355	valid-rmse:0.27246
[9546]	train-rmse:0.12356	valid-rmse:0.27239
[9547]	train-rmse:0.12356	valid-rmse:0.27244
[9548]	train-rmse:0.12357	valid-rmse:0.27250
[9549]	train-rmse:0.12357	valid-rmse:0.27242
[9550]	train-rmse:0.12356	valid-rmse:0.27249
[9551]	train-rmse:0.12356	valid-rmse:0.27248
[9552]	train-rmse:0.12355	valid-rmse:0.27251
[9553]	train-rmse:0.12355	valid-rmse:0.27245
[9554]	train-rmse:0.12354	valid-rmse:0.27248
[9555]	train-rmse:0.12354	valid-rmse:0.27243
[9556]	train-rmse:0.12354	valid-rmse:0.27253
[9557]	train-rmse:0.12354	valid-rmse:0.27251
[9558]	train-rmse:0.12353	valid-rmse:0.27266
[9559]	train-rmse:0.12354	valid-rmse:0.27266
[9560]	train-rmse:0.12354	valid-rmse:0.27266
[9561]	tra

[9722]	train-rmse:0.12355	valid-rmse:0.27257
[9723]	train-rmse:0.12356	valid-rmse:0.27259
[9724]	train-rmse:0.12356	valid-rmse:0.27268
[9725]	train-rmse:0.12355	valid-rmse:0.27280
[9726]	train-rmse:0.12355	valid-rmse:0.27280
[9727]	train-rmse:0.12356	valid-rmse:0.27288
[9728]	train-rmse:0.12356	valid-rmse:0.27291
[9729]	train-rmse:0.12356	valid-rmse:0.27287
[9730]	train-rmse:0.12357	valid-rmse:0.27284
[9731]	train-rmse:0.12357	valid-rmse:0.27281
[9732]	train-rmse:0.12356	valid-rmse:0.27276
[9733]	train-rmse:0.12357	valid-rmse:0.27276
[9734]	train-rmse:0.12356	valid-rmse:0.27283
[9735]	train-rmse:0.12354	valid-rmse:0.27281
[9736]	train-rmse:0.12354	valid-rmse:0.27277
[9737]	train-rmse:0.12353	valid-rmse:0.27284
[9738]	train-rmse:0.12353	valid-rmse:0.27295
[9739]	train-rmse:0.12354	valid-rmse:0.27285
[9740]	train-rmse:0.12353	valid-rmse:0.27291
[9741]	train-rmse:0.12352	valid-rmse:0.27296
[9742]	train-rmse:0.12352	valid-rmse:0.27302
[9743]	train-rmse:0.12352	valid-rmse:0.27311
[9744]	tra

[9905]	train-rmse:0.12350	valid-rmse:0.27307
[9906]	train-rmse:0.12350	valid-rmse:0.27308
[9907]	train-rmse:0.12350	valid-rmse:0.27305
[9908]	train-rmse:0.12349	valid-rmse:0.27308
[9909]	train-rmse:0.12349	valid-rmse:0.27312
[9910]	train-rmse:0.12349	valid-rmse:0.27314
[9911]	train-rmse:0.12349	valid-rmse:0.27315
[9912]	train-rmse:0.12349	valid-rmse:0.27321
[9913]	train-rmse:0.12349	valid-rmse:0.27317
[9914]	train-rmse:0.12350	valid-rmse:0.27304
[9915]	train-rmse:0.12349	valid-rmse:0.27303
[9916]	train-rmse:0.12350	valid-rmse:0.27304
[9917]	train-rmse:0.12351	valid-rmse:0.27292
[9918]	train-rmse:0.12352	valid-rmse:0.27285
[9919]	train-rmse:0.12352	valid-rmse:0.27269
[9920]	train-rmse:0.12353	valid-rmse:0.27274
[9921]	train-rmse:0.12352	valid-rmse:0.27273
[9922]	train-rmse:0.12352	valid-rmse:0.27253
[9923]	train-rmse:0.12352	valid-rmse:0.27240
[9924]	train-rmse:0.12352	valid-rmse:0.27248
[9925]	train-rmse:0.12353	valid-rmse:0.27230
[9926]	train-rmse:0.12353	valid-rmse:0.27224
[9927]	tra

In [150]:
pred_proba = obj_xgb.predict(dv_xgb)

pred = np.where(pred_proba > 0.055, 1, 0)

In [151]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
print(precision_score(pred, y_test))
print(recall_score(pred, y_test))
print(accuracy_score(pred, y_test))

0.22666666666666666
0.08629441624365482
0.8447488584474886


# Light GBM

In [152]:
prm_lgb = {
                        'task': 'train',
                        'boosting': 'goss',
                        'objective': 'binary',
                        'metric': 'rmse',
                        'learning_rate': 0.005,
                        'subsample': 0.9855232997390695,
                        'max_depth': 8,
                        'top_rate': 0.9064148448434349,
                        'num_leaves': 87,
                        'min_child_weight': 41.9612869171337,
                        'other_rate': 0.0721768246018207,
                        'reg_alpha': 9.677537745007898,
                        'colsample_bytree': 0.5665320670155495,
                        'min_split_gain': 9.820197773625843,
                        'reg_lambda': 8.2532317400459,
                        'min_data_in_leaf': 21,
                        'verbose': -1
                        }

In [90]:
import lightgbm as lgb
dt_lgb   = lgb.Dataset(x_train, y_train[:, 0], free_raw_data=False)
dv_lgb   = lgb.Dataset(x_test, y_test[:, 0], reference=dt_lgb, free_raw_data=False)

In [91]:
time_s = time.time()
obj_lgb = lgb.train(
    prm_lgb, dt_lgb, num_boost_round=10000,
    valid_sets=dv_lgb)
time_t = time.time()
print(time_t - time_s)

[1]	valid_0's rmse: 0.2157
[2]	valid_0's rmse: 0.215683
[3]	valid_0's rmse: 0.215657
[4]	valid_0's rmse: 0.215634
[5]	valid_0's rmse: 0.215585
[6]	valid_0's rmse: 0.215568
[7]	valid_0's rmse: 0.215519
[8]	valid_0's rmse: 0.215519
[9]	valid_0's rmse: 0.215511
[10]	valid_0's rmse: 0.215463
[11]	valid_0's rmse: 0.215446
[12]	valid_0's rmse: 0.215398
[13]	valid_0's rmse: 0.215381
[14]	valid_0's rmse: 0.215333
[15]	valid_0's rmse: 0.215333
[16]	valid_0's rmse: 0.215286
[17]	valid_0's rmse: 0.215239
[18]	valid_0's rmse: 0.215223
[19]	valid_0's rmse: 0.215176
[20]	valid_0's rmse: 0.21513
[21]	valid_0's rmse: 0.215084
[22]	valid_0's rmse: 0.215084
[23]	valid_0's rmse: 0.215084
[24]	valid_0's rmse: 0.215068
[25]	valid_0's rmse: 0.215068
[26]	valid_0's rmse: 0.215053
[27]	valid_0's rmse: 0.215007
[28]	valid_0's rmse: 0.214992
[29]	valid_0's rmse: 0.214977
[30]	valid_0's rmse: 0.214962
[31]	valid_0's rmse: 0.214917
[32]	valid_0's rmse: 0.214872
[33]	valid_0's rmse: 0.214857
[34]	valid_0's rmse: 0

[469]	valid_0's rmse: 0.211715
[470]	valid_0's rmse: 0.211715
[471]	valid_0's rmse: 0.211715
[472]	valid_0's rmse: 0.211715
[473]	valid_0's rmse: 0.211715
[474]	valid_0's rmse: 0.211715
[475]	valid_0's rmse: 0.211715
[476]	valid_0's rmse: 0.211715
[477]	valid_0's rmse: 0.211715
[478]	valid_0's rmse: 0.211715
[479]	valid_0's rmse: 0.211715
[480]	valid_0's rmse: 0.211715
[481]	valid_0's rmse: 0.211715
[482]	valid_0's rmse: 0.211715
[483]	valid_0's rmse: 0.211715
[484]	valid_0's rmse: 0.211715
[485]	valid_0's rmse: 0.211715
[486]	valid_0's rmse: 0.211715
[487]	valid_0's rmse: 0.211715
[488]	valid_0's rmse: 0.211715
[489]	valid_0's rmse: 0.211715
[490]	valid_0's rmse: 0.211715
[491]	valid_0's rmse: 0.211715
[492]	valid_0's rmse: 0.211715
[493]	valid_0's rmse: 0.211715
[494]	valid_0's rmse: 0.211715
[495]	valid_0's rmse: 0.211715
[496]	valid_0's rmse: 0.211715
[497]	valid_0's rmse: 0.211715
[498]	valid_0's rmse: 0.211715
[499]	valid_0's rmse: 0.211715
[500]	valid_0's rmse: 0.211715
[501]	va

[791]	valid_0's rmse: 0.211715
[792]	valid_0's rmse: 0.211715
[793]	valid_0's rmse: 0.211715
[794]	valid_0's rmse: 0.211715
[795]	valid_0's rmse: 0.211715
[796]	valid_0's rmse: 0.211715
[797]	valid_0's rmse: 0.211715
[798]	valid_0's rmse: 0.211715
[799]	valid_0's rmse: 0.211715
[800]	valid_0's rmse: 0.211715
[801]	valid_0's rmse: 0.211715
[802]	valid_0's rmse: 0.211715
[803]	valid_0's rmse: 0.211715
[804]	valid_0's rmse: 0.211715
[805]	valid_0's rmse: 0.211715
[806]	valid_0's rmse: 0.211715
[807]	valid_0's rmse: 0.211715
[808]	valid_0's rmse: 0.211715
[809]	valid_0's rmse: 0.211715
[810]	valid_0's rmse: 0.211715
[811]	valid_0's rmse: 0.211715
[812]	valid_0's rmse: 0.211715
[813]	valid_0's rmse: 0.211715
[814]	valid_0's rmse: 0.211715
[815]	valid_0's rmse: 0.211715
[816]	valid_0's rmse: 0.211715
[817]	valid_0's rmse: 0.211715
[818]	valid_0's rmse: 0.211715
[819]	valid_0's rmse: 0.211715
[820]	valid_0's rmse: 0.211715
[821]	valid_0's rmse: 0.211715
[822]	valid_0's rmse: 0.211715
[823]	va

[1106]	valid_0's rmse: 0.211715
[1107]	valid_0's rmse: 0.211715
[1108]	valid_0's rmse: 0.211715
[1109]	valid_0's rmse: 0.211715
[1110]	valid_0's rmse: 0.211715
[1111]	valid_0's rmse: 0.211715
[1112]	valid_0's rmse: 0.211715
[1113]	valid_0's rmse: 0.211715
[1114]	valid_0's rmse: 0.211715
[1115]	valid_0's rmse: 0.211715
[1116]	valid_0's rmse: 0.211715
[1117]	valid_0's rmse: 0.211715
[1118]	valid_0's rmse: 0.211715
[1119]	valid_0's rmse: 0.211715
[1120]	valid_0's rmse: 0.211715
[1121]	valid_0's rmse: 0.211715
[1122]	valid_0's rmse: 0.211715
[1123]	valid_0's rmse: 0.211715
[1124]	valid_0's rmse: 0.211715
[1125]	valid_0's rmse: 0.211715
[1126]	valid_0's rmse: 0.211715
[1127]	valid_0's rmse: 0.211715
[1128]	valid_0's rmse: 0.211715
[1129]	valid_0's rmse: 0.211715
[1130]	valid_0's rmse: 0.211715
[1131]	valid_0's rmse: 0.211715
[1132]	valid_0's rmse: 0.211715
[1133]	valid_0's rmse: 0.211715
[1134]	valid_0's rmse: 0.211715
[1135]	valid_0's rmse: 0.211715
[1136]	valid_0's rmse: 0.211715
[1137]	v

[1426]	valid_0's rmse: 0.211715
[1427]	valid_0's rmse: 0.211715
[1428]	valid_0's rmse: 0.211715
[1429]	valid_0's rmse: 0.211715
[1430]	valid_0's rmse: 0.211715
[1431]	valid_0's rmse: 0.211715
[1432]	valid_0's rmse: 0.211715
[1433]	valid_0's rmse: 0.211715
[1434]	valid_0's rmse: 0.211715
[1435]	valid_0's rmse: 0.211715
[1436]	valid_0's rmse: 0.211715
[1437]	valid_0's rmse: 0.211715
[1438]	valid_0's rmse: 0.211715
[1439]	valid_0's rmse: 0.211715
[1440]	valid_0's rmse: 0.211715
[1441]	valid_0's rmse: 0.211715
[1442]	valid_0's rmse: 0.211715
[1443]	valid_0's rmse: 0.211715
[1444]	valid_0's rmse: 0.211715
[1445]	valid_0's rmse: 0.211715
[1446]	valid_0's rmse: 0.211715
[1447]	valid_0's rmse: 0.211715
[1448]	valid_0's rmse: 0.211715
[1449]	valid_0's rmse: 0.211715
[1450]	valid_0's rmse: 0.211715
[1451]	valid_0's rmse: 0.211715
[1452]	valid_0's rmse: 0.211715
[1453]	valid_0's rmse: 0.211715
[1454]	valid_0's rmse: 0.211715
[1455]	valid_0's rmse: 0.211715
[1456]	valid_0's rmse: 0.211715
[1457]	v

...

[8534]	valid_0's rmse: 0.211715
[8535]	valid_0's rmse: 0.211715
[8536]	valid_0's rmse: 0.211715
[8537]	valid_0's rmse: 0.211715
[8538]	valid_0's rmse: 0.211715
[8539]	valid_0's rmse: 0.211715
[8540]	valid_0's rmse: 0.211715
[8541]	valid_0's rmse: 0.211715
[8542]	valid_0's rmse: 0.211715
[8543]	valid_0's rmse: 0.211715
[8544]	valid_0's rmse: 0.211715
[8545]	valid_0's rmse: 0.211715
[8546]	valid_0's rmse: 0.211715
[8547]	valid_0's rmse: 0.211715
[8548]	valid_0's rmse: 0.211715
[8549]	valid_0's rmse: 0.211715
[8550]	valid_0's rmse: 0.211715
[8551]	valid_0's rmse: 0.211715
[8552]	valid_0's rmse: 0.211715
[8553]	valid_0's rmse: 0.211715
[8554]	valid_0's rmse: 0.211715
[8555]	valid_0's rmse: 0.211715
[8556]	valid_0's rmse: 0.211715
[8557]	valid_0's rmse: 0.211715
[8558]	valid_0's rmse: 0.211715
[8559]	valid_0's rmse: 0.211715
[8560]	valid_0's rmse: 0.211715
[8561]	valid_0's rmse: 0.211715
[8562]	valid_0's rmse: 0.211715
[8563]	valid_0's rmse: 0.211715
[8564]	valid_0's rmse: 0.211715
[8565]	v

[8883]	valid_0's rmse: 0.211715
[8884]	valid_0's rmse: 0.211715
[8885]	valid_0's rmse: 0.211715
[8886]	valid_0's rmse: 0.211715
[8887]	valid_0's rmse: 0.211715
[8888]	valid_0's rmse: 0.211715
[8889]	valid_0's rmse: 0.211715
[8890]	valid_0's rmse: 0.211715
[8891]	valid_0's rmse: 0.211715
[8892]	valid_0's rmse: 0.211715
[8893]	valid_0's rmse: 0.211715
[8894]	valid_0's rmse: 0.211715
[8895]	valid_0's rmse: 0.211715
[8896]	valid_0's rmse: 0.211715
[8897]	valid_0's rmse: 0.211715
[8898]	valid_0's rmse: 0.211715
[8899]	valid_0's rmse: 0.211715
[8900]	valid_0's rmse: 0.211715
[8901]	valid_0's rmse: 0.211715
[8902]	valid_0's rmse: 0.211715
[8903]	valid_0's rmse: 0.211715
[8904]	valid_0's rmse: 0.211715
[8905]	valid_0's rmse: 0.211715
[8906]	valid_0's rmse: 0.211715
[8907]	valid_0's rmse: 0.211715
[8908]	valid_0's rmse: 0.211715
[8909]	valid_0's rmse: 0.211715
[8910]	valid_0's rmse: 0.211715
[8911]	valid_0's rmse: 0.211715
[8912]	valid_0's rmse: 0.211715
[8913]	valid_0's rmse: 0.211715
[8914]	v

[9207]	valid_0's rmse: 0.211715
[9208]	valid_0's rmse: 0.211715
[9209]	valid_0's rmse: 0.211715
[9210]	valid_0's rmse: 0.211715
[9211]	valid_0's rmse: 0.211715
[9212]	valid_0's rmse: 0.211715
[9213]	valid_0's rmse: 0.211715
[9214]	valid_0's rmse: 0.211715
[9215]	valid_0's rmse: 0.211715
[9216]	valid_0's rmse: 0.211715
[9217]	valid_0's rmse: 0.211715
[9218]	valid_0's rmse: 0.211715
[9219]	valid_0's rmse: 0.211715
[9220]	valid_0's rmse: 0.211715
[9221]	valid_0's rmse: 0.211715
[9222]	valid_0's rmse: 0.211715
[9223]	valid_0's rmse: 0.211715
[9224]	valid_0's rmse: 0.211715
[9225]	valid_0's rmse: 0.211715
[9226]	valid_0's rmse: 0.211715
[9227]	valid_0's rmse: 0.211715
[9228]	valid_0's rmse: 0.211715
[9229]	valid_0's rmse: 0.211715
[9230]	valid_0's rmse: 0.211715
[9231]	valid_0's rmse: 0.211715
[9232]	valid_0's rmse: 0.211715
[9233]	valid_0's rmse: 0.211715
[9234]	valid_0's rmse: 0.211715
[9235]	valid_0's rmse: 0.211715
[9236]	valid_0's rmse: 0.211715
[9237]	valid_0's rmse: 0.211715
[9238]	v

[9536]	valid_0's rmse: 0.211715
[9537]	valid_0's rmse: 0.211715
[9538]	valid_0's rmse: 0.211715
[9539]	valid_0's rmse: 0.211715
[9540]	valid_0's rmse: 0.211715
[9541]	valid_0's rmse: 0.211715
[9542]	valid_0's rmse: 0.211715
[9543]	valid_0's rmse: 0.211715
[9544]	valid_0's rmse: 0.211715
[9545]	valid_0's rmse: 0.211715
[9546]	valid_0's rmse: 0.211715
[9547]	valid_0's rmse: 0.211715
[9548]	valid_0's rmse: 0.211715
[9549]	valid_0's rmse: 0.211715
[9550]	valid_0's rmse: 0.211715
[9551]	valid_0's rmse: 0.211715
[9552]	valid_0's rmse: 0.211715
[9553]	valid_0's rmse: 0.211715
[9554]	valid_0's rmse: 0.211715
[9555]	valid_0's rmse: 0.211715
[9556]	valid_0's rmse: 0.211715
[9557]	valid_0's rmse: 0.211715
[9558]	valid_0's rmse: 0.211715
[9559]	valid_0's rmse: 0.211715
[9560]	valid_0's rmse: 0.211715
[9561]	valid_0's rmse: 0.211715
[9562]	valid_0's rmse: 0.211715
[9563]	valid_0's rmse: 0.211715
[9564]	valid_0's rmse: 0.211715
[9565]	valid_0's rmse: 0.211715
[9566]	valid_0's rmse: 0.211715
[9567]	v

[9883]	valid_0's rmse: 0.211715
[9884]	valid_0's rmse: 0.211715
[9885]	valid_0's rmse: 0.211715
[9886]	valid_0's rmse: 0.211715
[9887]	valid_0's rmse: 0.211715
[9888]	valid_0's rmse: 0.211715
[9889]	valid_0's rmse: 0.211715
[9890]	valid_0's rmse: 0.211715
[9891]	valid_0's rmse: 0.211715
[9892]	valid_0's rmse: 0.211715
[9893]	valid_0's rmse: 0.211715
[9894]	valid_0's rmse: 0.211715
[9895]	valid_0's rmse: 0.211715
[9896]	valid_0's rmse: 0.211715
[9897]	valid_0's rmse: 0.211715
[9898]	valid_0's rmse: 0.211715
[9899]	valid_0's rmse: 0.211715
[9900]	valid_0's rmse: 0.211715
[9901]	valid_0's rmse: 0.211715
[9902]	valid_0's rmse: 0.211715
[9903]	valid_0's rmse: 0.211715
[9904]	valid_0's rmse: 0.211715
[9905]	valid_0's rmse: 0.211715
[9906]	valid_0's rmse: 0.211715
[9907]	valid_0's rmse: 0.211715
[9908]	valid_0's rmse: 0.211715
[9909]	valid_0's rmse: 0.211715
[9910]	valid_0's rmse: 0.211715
[9911]	valid_0's rmse: 0.211715
[9912]	valid_0's rmse: 0.211715
[9913]	valid_0's rmse: 0.211715
[9914]	v

In [134]:
pred_proba = obj_lgb.predict(x_test)

pred = np.where(pred_proba > 0.041, 1, 0)

In [135]:
print(precision_score(pred, y_test))
print(recall_score(pred, y_test))
print(accuracy_score(pred, y_test))

0.9066666666666666
0.09497206703910614
0.5727332028701891


In [191]:
from sklearn.model_selection import KFold, StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#X = X.to_numpy()
#y = y.to_numpy()

precision_scores = []
recall_scores = []
predictions = []
data_y = []
for train_index, test_index in kf.split(X, y):
    
    dt_xgb = xgb.DMatrix(X[train_index], y[train_index])
    dv_xgb = xgb.DMatrix(X[test_index], y[test_index])
    
    training = xgb.train(
    prm_xgb, dt_xgb, 100,
    [(dt_xgb, 'train'), (dv_xgb, 'valid')])
    
    pred_proba = training.predict(dv_xgb)
    pred = np.where(pred_proba > 0.055, 1, 0)
    
    precision_scores.append(
        precision_score(pred, y[test_index][:, 0])
    )
    recall_scores.append(
        recall_score(pred, y[test_index][:, 0])
    )

[0]	train-rmse:0.45942	valid-rmse:0.45952
[1]	train-rmse:0.42386	valid-rmse:0.42392
[2]	train-rmse:0.39265	valid-rmse:0.39285
[3]	train-rmse:0.36530	valid-rmse:0.36558
[4]	train-rmse:0.34160	valid-rmse:0.34194
[5]	train-rmse:0.32088	valid-rmse:0.32128
[6]	train-rmse:0.30283	valid-rmse:0.30340
[7]	train-rmse:0.28713	valid-rmse:0.28781
[8]	train-rmse:0.27394	valid-rmse:0.27469
[9]	train-rmse:0.26265	valid-rmse:0.26369
[10]	train-rmse:0.25295	valid-rmse:0.25399
[11]	train-rmse:0.24468	valid-rmse:0.24587
[12]	train-rmse:0.23783	valid-rmse:0.23922
[13]	train-rmse:0.23209	valid-rmse:0.23363
[14]	train-rmse:0.22708	valid-rmse:0.22893
[15]	train-rmse:0.22290	valid-rmse:0.22498
[16]	train-rmse:0.21935	valid-rmse:0.22187
[17]	train-rmse:0.21644	valid-rmse:0.21927
[18]	train-rmse:0.21400	valid-rmse:0.21721
[19]	train-rmse:0.21196	valid-rmse:0.21550
[20]	train-rmse:0.21021	valid-rmse:0.21401
[21]	train-rmse:0.20881	valid-rmse:0.21284
[22]	train-rmse:0.20761	valid-rmse:0.21178
[23]	train-rmse:0.206

[91]	train-rmse:0.18980	valid-rmse:0.21435
[92]	train-rmse:0.18954	valid-rmse:0.21442
[93]	train-rmse:0.18935	valid-rmse:0.21428
[94]	train-rmse:0.18921	valid-rmse:0.21427
[95]	train-rmse:0.18902	valid-rmse:0.21434
[96]	train-rmse:0.18885	valid-rmse:0.21433
[97]	train-rmse:0.18872	valid-rmse:0.21431
[98]	train-rmse:0.18853	valid-rmse:0.21421
[99]	train-rmse:0.18845	valid-rmse:0.21419
[0]	train-rmse:0.45934	valid-rmse:0.45923
[1]	train-rmse:0.42360	valid-rmse:0.42387
[2]	train-rmse:0.39239	valid-rmse:0.39295
[3]	train-rmse:0.36510	valid-rmse:0.36582
[4]	train-rmse:0.34138	valid-rmse:0.34252
[5]	train-rmse:0.32059	valid-rmse:0.32214
[6]	train-rmse:0.30245	valid-rmse:0.30421
[7]	train-rmse:0.28680	valid-rmse:0.28898
[8]	train-rmse:0.27347	valid-rmse:0.27622
[9]	train-rmse:0.26219	valid-rmse:0.26522
[10]	train-rmse:0.25244	valid-rmse:0.25581
[11]	train-rmse:0.24418	valid-rmse:0.24796
[12]	train-rmse:0.23731	valid-rmse:0.24134
[13]	train-rmse:0.23156	valid-rmse:0.23584
[14]	train-rmse:0.226

[82]	train-rmse:0.19317	valid-rmse:0.21025
[83]	train-rmse:0.19291	valid-rmse:0.21035
[84]	train-rmse:0.19271	valid-rmse:0.21049
[85]	train-rmse:0.19246	valid-rmse:0.21038
[86]	train-rmse:0.19235	valid-rmse:0.21043
[87]	train-rmse:0.19223	valid-rmse:0.21053
[88]	train-rmse:0.19210	valid-rmse:0.21046
[89]	train-rmse:0.19184	valid-rmse:0.21072
[90]	train-rmse:0.19163	valid-rmse:0.21071
[91]	train-rmse:0.19149	valid-rmse:0.21084
[92]	train-rmse:0.19141	valid-rmse:0.21063
[93]	train-rmse:0.19118	valid-rmse:0.21090
[94]	train-rmse:0.19098	valid-rmse:0.21100
[95]	train-rmse:0.19080	valid-rmse:0.21102
[96]	train-rmse:0.19059	valid-rmse:0.21105
[97]	train-rmse:0.19040	valid-rmse:0.21108
[98]	train-rmse:0.19019	valid-rmse:0.21115
[99]	train-rmse:0.19000	valid-rmse:0.21129
[0]	train-rmse:0.45930	valid-rmse:0.45953
[1]	train-rmse:0.42367	valid-rmse:0.42419
[2]	train-rmse:0.39229	valid-rmse:0.39318
[3]	train-rmse:0.36491	valid-rmse:0.36622
[4]	train-rmse:0.34119	valid-rmse:0.34304
[5]	train-rmse:0

In [192]:
print("Precision Score:  ", np.mean(precision_scores), "\n\nRecall Score:  ", np.mean(recall_scores))

Precision Score:   0.6863673469387755 

Recall Score:   0.11434115246794334


In [195]:
from sklearn.model_selection import KFold, StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#X = X.to_numpy()
#y = y.to_numpy()

precision_scores = []
recall_scores = []
predictions = []
data_y = []
for train_index, test_index in kf.split(X, y):
    
    dt_lgb = lgb.Dataset(X[train_index], y[train_index][:,0])
    dv_lgb = lgb.Dataset(X[test_index], y[test_index][:,0])
    
    training = lgb.train(
    prm_lgb, dt_lgb, num_boost_round=1000,
    valid_sets=dv_lgb)
    
    pred_proba = training.predict(X[test_index])
    pred = np.where(pred_proba > 0.055, 1, 0)
    
    precision_scores.append(
        precision_score(pred, y[test_index][:, 0])
    )
    recall_scores.append(
        recall_score(pred, y[test_index][:, 0])
    )

[1]	valid_0's rmse: 0.215702
[2]	valid_0's rmse: 0.215679
[3]	valid_0's rmse: 0.215632
[4]	valid_0's rmse: 0.215585
[5]	valid_0's rmse: 0.215539
[6]	valid_0's rmse: 0.215516
[7]	valid_0's rmse: 0.21547
[8]	valid_0's rmse: 0.21547
[9]	valid_0's rmse: 0.215464
[10]	valid_0's rmse: 0.215418
[11]	valid_0's rmse: 0.215396
[12]	valid_0's rmse: 0.215351
[13]	valid_0's rmse: 0.215329
[14]	valid_0's rmse: 0.215284
[15]	valid_0's rmse: 0.215284
[16]	valid_0's rmse: 0.21524
[17]	valid_0's rmse: 0.215196
[18]	valid_0's rmse: 0.215175
[19]	valid_0's rmse: 0.215131
[20]	valid_0's rmse: 0.215088
[21]	valid_0's rmse: 0.215045
[22]	valid_0's rmse: 0.215045
[23]	valid_0's rmse: 0.215045
[24]	valid_0's rmse: 0.215024
[25]	valid_0's rmse: 0.215024
[26]	valid_0's rmse: 0.215003
[27]	valid_0's rmse: 0.214961
[28]	valid_0's rmse: 0.21494
[29]	valid_0's rmse: 0.21492
[30]	valid_0's rmse: 0.2149
[31]	valid_0's rmse: 0.214858
[32]	valid_0's rmse: 0.214816
[33]	valid_0's rmse: 0.214797
[34]	valid_0's rmse: 0.214

[367]	valid_0's rmse: 0.21126
[368]	valid_0's rmse: 0.21126
[369]	valid_0's rmse: 0.21126
[370]	valid_0's rmse: 0.21126
[371]	valid_0's rmse: 0.21126
[372]	valid_0's rmse: 0.21126
[373]	valid_0's rmse: 0.21126
[374]	valid_0's rmse: 0.21126
[375]	valid_0's rmse: 0.21126
[376]	valid_0's rmse: 0.21126
[377]	valid_0's rmse: 0.21126
[378]	valid_0's rmse: 0.21126
[379]	valid_0's rmse: 0.21126
[380]	valid_0's rmse: 0.21126
[381]	valid_0's rmse: 0.21126
[382]	valid_0's rmse: 0.21126
[383]	valid_0's rmse: 0.21126
[384]	valid_0's rmse: 0.21126
[385]	valid_0's rmse: 0.21126
[386]	valid_0's rmse: 0.21126
[387]	valid_0's rmse: 0.21126
[388]	valid_0's rmse: 0.21126
[389]	valid_0's rmse: 0.21126
[390]	valid_0's rmse: 0.21126
[391]	valid_0's rmse: 0.21126
[392]	valid_0's rmse: 0.21126
[393]	valid_0's rmse: 0.21126
[394]	valid_0's rmse: 0.21126
[395]	valid_0's rmse: 0.21126
[396]	valid_0's rmse: 0.21126
[397]	valid_0's rmse: 0.21126
[398]	valid_0's rmse: 0.21126
[399]	valid_0's rmse: 0.21126
[400]	vali

[688]	valid_0's rmse: 0.21126
[689]	valid_0's rmse: 0.21126
[690]	valid_0's rmse: 0.21126
[691]	valid_0's rmse: 0.21126
[692]	valid_0's rmse: 0.21126
[693]	valid_0's rmse: 0.21126
[694]	valid_0's rmse: 0.21126
[695]	valid_0's rmse: 0.21126
[696]	valid_0's rmse: 0.21126
[697]	valid_0's rmse: 0.21126
[698]	valid_0's rmse: 0.21126
[699]	valid_0's rmse: 0.21126
[700]	valid_0's rmse: 0.21126
[701]	valid_0's rmse: 0.21126
[702]	valid_0's rmse: 0.21126
[703]	valid_0's rmse: 0.21126
[704]	valid_0's rmse: 0.21126
[705]	valid_0's rmse: 0.21126
[706]	valid_0's rmse: 0.21126
[707]	valid_0's rmse: 0.21126
[708]	valid_0's rmse: 0.21126
[709]	valid_0's rmse: 0.21126
[710]	valid_0's rmse: 0.21126
[711]	valid_0's rmse: 0.21126
[712]	valid_0's rmse: 0.21126
[713]	valid_0's rmse: 0.21126
[714]	valid_0's rmse: 0.21126
[715]	valid_0's rmse: 0.21126
[716]	valid_0's rmse: 0.21126
[717]	valid_0's rmse: 0.21126
[718]	valid_0's rmse: 0.21126
[719]	valid_0's rmse: 0.21126
[720]	valid_0's rmse: 0.21126
[721]	vali

[1]	valid_0's rmse: 0.215699
[2]	valid_0's rmse: 0.215681
[3]	valid_0's rmse: 0.21563
[4]	valid_0's rmse: 0.21558
[5]	valid_0's rmse: 0.215531
[6]	valid_0's rmse: 0.215513
[7]	valid_0's rmse: 0.215464
[8]	valid_0's rmse: 0.215464
[9]	valid_0's rmse: 0.215454
[10]	valid_0's rmse: 0.215406
[11]	valid_0's rmse: 0.215389
[12]	valid_0's rmse: 0.21534
[13]	valid_0's rmse: 0.215323
[14]	valid_0's rmse: 0.215276
[15]	valid_0's rmse: 0.215276
[16]	valid_0's rmse: 0.215228
[17]	valid_0's rmse: 0.215181
[18]	valid_0's rmse: 0.215165
[19]	valid_0's rmse: 0.215118
[20]	valid_0's rmse: 0.215072
[21]	valid_0's rmse: 0.215026
[22]	valid_0's rmse: 0.215026
[23]	valid_0's rmse: 0.215026
[24]	valid_0's rmse: 0.21501
[25]	valid_0's rmse: 0.21501
[26]	valid_0's rmse: 0.214994
[27]	valid_0's rmse: 0.214948
[28]	valid_0's rmse: 0.214933
[29]	valid_0's rmse: 0.214917
[30]	valid_0's rmse: 0.214902
[31]	valid_0's rmse: 0.214857
[32]	valid_0's rmse: 0.214812
[33]	valid_0's rmse: 0.214797
[34]	valid_0's rmse: 0.2

[293]	valid_0's rmse: 0.211049
[294]	valid_0's rmse: 0.211046
[295]	valid_0's rmse: 0.211043
[296]	valid_0's rmse: 0.211043
[297]	valid_0's rmse: 0.211043
[298]	valid_0's rmse: 0.211043
[299]	valid_0's rmse: 0.211043
[300]	valid_0's rmse: 0.21104
[301]	valid_0's rmse: 0.21104
[302]	valid_0's rmse: 0.21104
[303]	valid_0's rmse: 0.211038
[304]	valid_0's rmse: 0.211038
[305]	valid_0's rmse: 0.211038
[306]	valid_0's rmse: 0.211038
[307]	valid_0's rmse: 0.211038
[308]	valid_0's rmse: 0.211038
[309]	valid_0's rmse: 0.211038
[310]	valid_0's rmse: 0.211038
[311]	valid_0's rmse: 0.211038
[312]	valid_0's rmse: 0.211038
[313]	valid_0's rmse: 0.211038
[314]	valid_0's rmse: 0.211038
[315]	valid_0's rmse: 0.211038
[316]	valid_0's rmse: 0.211038
[317]	valid_0's rmse: 0.211038
[318]	valid_0's rmse: 0.211038
[319]	valid_0's rmse: 0.211038
[320]	valid_0's rmse: 0.211038
[321]	valid_0's rmse: 0.211038
[322]	valid_0's rmse: 0.211038
[323]	valid_0's rmse: 0.211038
[324]	valid_0's rmse: 0.211038
[325]	valid

...

[604]	valid_0's rmse: 0.210766
[605]	valid_0's rmse: 0.210766
[606]	valid_0's rmse: 0.210766
[607]	valid_0's rmse: 0.210766
[608]	valid_0's rmse: 0.210766
[609]	valid_0's rmse: 0.210766
[610]	valid_0's rmse: 0.210766
[611]	valid_0's rmse: 0.210766
[612]	valid_0's rmse: 0.210766
[613]	valid_0's rmse: 0.210766
[614]	valid_0's rmse: 0.210766
[615]	valid_0's rmse: 0.210766
[616]	valid_0's rmse: 0.210766
[617]	valid_0's rmse: 0.210766
[618]	valid_0's rmse: 0.210766
[619]	valid_0's rmse: 0.210766
[620]	valid_0's rmse: 0.210766
[621]	valid_0's rmse: 0.210766
[622]	valid_0's rmse: 0.210766
[623]	valid_0's rmse: 0.210766
[624]	valid_0's rmse: 0.210766
[625]	valid_0's rmse: 0.210766
[626]	valid_0's rmse: 0.210766
[627]	valid_0's rmse: 0.210766
[628]	valid_0's rmse: 0.210766
[629]	valid_0's rmse: 0.210766
[630]	valid_0's rmse: 0.210766
[631]	valid_0's rmse: 0.210766
[632]	valid_0's rmse: 0.210766
[633]	valid_0's rmse: 0.210766
[634]	valid_0's rmse: 0.210766
[635]	valid_0's rmse: 0.210766
[636]	va

[949]	valid_0's rmse: 0.210766
[950]	valid_0's rmse: 0.210766
[951]	valid_0's rmse: 0.210766
[952]	valid_0's rmse: 0.210766
[953]	valid_0's rmse: 0.210766
[954]	valid_0's rmse: 0.210766
[955]	valid_0's rmse: 0.210766
[956]	valid_0's rmse: 0.210766
[957]	valid_0's rmse: 0.210766
[958]	valid_0's rmse: 0.210766
[959]	valid_0's rmse: 0.210766
[960]	valid_0's rmse: 0.210766
[961]	valid_0's rmse: 0.210766
[962]	valid_0's rmse: 0.210766
[963]	valid_0's rmse: 0.210766
[964]	valid_0's rmse: 0.210766
[965]	valid_0's rmse: 0.210766
[966]	valid_0's rmse: 0.210766
[967]	valid_0's rmse: 0.210766
[968]	valid_0's rmse: 0.210766
[969]	valid_0's rmse: 0.210766
[970]	valid_0's rmse: 0.210766
[971]	valid_0's rmse: 0.210766
[972]	valid_0's rmse: 0.210766
[973]	valid_0's rmse: 0.210766
[974]	valid_0's rmse: 0.210766
[975]	valid_0's rmse: 0.210766
[976]	valid_0's rmse: 0.210766
[977]	valid_0's rmse: 0.210766
[978]	valid_0's rmse: 0.210766
[979]	valid_0's rmse: 0.210766
[980]	valid_0's rmse: 0.210766
[981]	va

[342]	valid_0's rmse: 0.209892
[343]	valid_0's rmse: 0.209892
[344]	valid_0's rmse: 0.209892
[345]	valid_0's rmse: 0.209892
[346]	valid_0's rmse: 0.209892
[347]	valid_0's rmse: 0.209892
[348]	valid_0's rmse: 0.209892
[349]	valid_0's rmse: 0.209892
[350]	valid_0's rmse: 0.209892
[351]	valid_0's rmse: 0.209892
[352]	valid_0's rmse: 0.209892
[353]	valid_0's rmse: 0.209892
[354]	valid_0's rmse: 0.209892
[355]	valid_0's rmse: 0.209892
[356]	valid_0's rmse: 0.209892
[357]	valid_0's rmse: 0.209892
[358]	valid_0's rmse: 0.209892
[359]	valid_0's rmse: 0.209892
[360]	valid_0's rmse: 0.209892
[361]	valid_0's rmse: 0.209892
[362]	valid_0's rmse: 0.209892
[363]	valid_0's rmse: 0.209892
[364]	valid_0's rmse: 0.209892
[365]	valid_0's rmse: 0.209892
[366]	valid_0's rmse: 0.209892
[367]	valid_0's rmse: 0.209892
[368]	valid_0's rmse: 0.209892
[369]	valid_0's rmse: 0.209892
[370]	valid_0's rmse: 0.209892
[371]	valid_0's rmse: 0.209892
[372]	valid_0's rmse: 0.209892
[373]	valid_0's rmse: 0.209892
[374]	va

[691]	valid_0's rmse: 0.209892
[692]	valid_0's rmse: 0.209892
[693]	valid_0's rmse: 0.209892
[694]	valid_0's rmse: 0.209892
[695]	valid_0's rmse: 0.209892
[696]	valid_0's rmse: 0.209892
[697]	valid_0's rmse: 0.209892
[698]	valid_0's rmse: 0.209892
[699]	valid_0's rmse: 0.209892
[700]	valid_0's rmse: 0.209892
[701]	valid_0's rmse: 0.209892
[702]	valid_0's rmse: 0.209892
[703]	valid_0's rmse: 0.209892
[704]	valid_0's rmse: 0.209892
[705]	valid_0's rmse: 0.209892
[706]	valid_0's rmse: 0.209892
[707]	valid_0's rmse: 0.209892
[708]	valid_0's rmse: 0.209892
[709]	valid_0's rmse: 0.209892
[710]	valid_0's rmse: 0.209892
[711]	valid_0's rmse: 0.209892
[712]	valid_0's rmse: 0.209892
[713]	valid_0's rmse: 0.209892
[714]	valid_0's rmse: 0.209892
[715]	valid_0's rmse: 0.209892
[716]	valid_0's rmse: 0.209892
[717]	valid_0's rmse: 0.209892
[718]	valid_0's rmse: 0.209892
[719]	valid_0's rmse: 0.209892
[720]	valid_0's rmse: 0.209892
[721]	valid_0's rmse: 0.209892
[722]	valid_0's rmse: 0.209892
[723]	va

[973]	valid_0's rmse: 0.209892
[974]	valid_0's rmse: 0.209892
[975]	valid_0's rmse: 0.209892
[976]	valid_0's rmse: 0.209892
[977]	valid_0's rmse: 0.209892
[978]	valid_0's rmse: 0.209892
[979]	valid_0's rmse: 0.209892
[980]	valid_0's rmse: 0.209892
[981]	valid_0's rmse: 0.209892
[982]	valid_0's rmse: 0.209892
[983]	valid_0's rmse: 0.209892
[984]	valid_0's rmse: 0.209892
[985]	valid_0's rmse: 0.209892
[986]	valid_0's rmse: 0.209892
[987]	valid_0's rmse: 0.209892
[988]	valid_0's rmse: 0.209892
[989]	valid_0's rmse: 0.209892
[990]	valid_0's rmse: 0.209892
[991]	valid_0's rmse: 0.209892
[992]	valid_0's rmse: 0.209892
[993]	valid_0's rmse: 0.209892
[994]	valid_0's rmse: 0.209892
[995]	valid_0's rmse: 0.209892
[996]	valid_0's rmse: 0.209892
[997]	valid_0's rmse: 0.209892
[998]	valid_0's rmse: 0.209892
[999]	valid_0's rmse: 0.209892
[1000]	valid_0's rmse: 0.209892


In [196]:
print("Precision Score:  ", np.mean(precision_scores), "\n\nRecall Score:  ", np.mean(recall_scores))

Precision Score:   0.7664489795918368 

Recall Score:   0.1184403247740905
